<a href="https://colab.research.google.com/github/jbiller123/Twirling-Orange-Northerners/blob/master/Part3_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 1

# Install any missing libraries

In [0]:
pip install sklearn

Note: you may need to restart the kernel to use updated packages.


In [0]:
pip install panda

Note: you may need to restart the kernel to use updated packages.


In [0]:
pip install fancyimpute

Note: you may need to restart the kernel to use updated packages.


In [0]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [0]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


# Import all libraries

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import Imputer  
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import mean_squared_error

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.impute import SimpleImputer
from sklearn.ensemble import VotingRegressor#create a dictionary of our models

##from fancyimpute import MICE

from sklearn.ensemble import AdaBoostRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import GridSearchCV
from sklearn import tree
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split


%matplotlib inline

In [0]:
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV #Importing RandomizedSearchCV for random forest regressor

# Load datasets

In [0]:
train_df = pd.read_csv('https://grantmlong.com/data/SE_rents2018_train.csv', index_col=0)
test_df = pd.read_csv('https://grantmlong.com/data/SE_rents2018_test1.csv', index_col=0)
submit2_df = pd.read_csv('https://grantmlong.com/data/SE_rents2018_test3.csv', index_col=0)
zips_df = pd.read_csv('https://raw.githubusercontent.com/jbiller123/Twirling-Orange-Northerners/master/zip_borough.csv', header=1, index_col=None, names=['zip','borough'])
ext_df = pd.read_csv('https://raw.githubusercontent.com/jbiller123/Twirling-Orange-Northerners/master/cleaned_external.csv',index_col=None)

# Merge External Dataset with Training and Testing Datasets

In [347]:
display(ext_df)

train_df=train_df.merge(ext_df, how='outer',right_index=True, on = 'borough')
test_df=test_df.merge(ext_df, how='outer',right_index=True, on = 'borough')
submit2_df=submit2_df.merge(ext_df, how='outer',right_index=True, on = 'borough')

,borough,population,num_units,building_val,gross_rent,retail_sales,mean_travel_time,median_in come,availailable_land
0,Manhattan,1586360,886249,915300,1615,27200,31.8,79781,22.83
1,Bronx,1384603,532487,371800,1133,4880,44.2,36593,42.10
2,Queens,2230578,865878,481300,1456,7481,43.5,62008,108.53
3,Brooklyn,2504717,1053767,623900,1314,8003,42.4,52782,70.82
4,Staten Island,468730,181199,460200,1229,8106,44.0,76244,58.37


#### Build training data

In [0]:
# extract usable features
feature_cols = [
    'bedrooms', 'bathrooms', 'floor_count', 'size_sqft',
    'has_washer_dryer', 'has_roofdeck', 'has_doorman', 'has_dishwasher','gross_rent', 	
    'retail_sales', 	'mean_travel_time', 	'median_in come', 	'availailable_land']
train_features = train_df[feature_cols] 

# impute missing values with imputer
imp = SimpleImputer(missing_values=np.nan, strategy='median')
imp.fit(train_features)
imp.transform(train_features)

# construct target vector
train_target = train_df['rent']



#### Fit models for training data

In [354]:
lreg = LinearRegression()
lreg.fit(train_features, train_target)

rf = RandomForestRegressor()
rf.fit(train_features, train_target)

br = BaggingRegressor(tree.DecisionTreeRegressor(random_state=1))
br.fit(train_features, train_target)

gb = GradientBoostingRegressor()
gb.fit(train_features, train_target)

ab = AdaBoostRegressor()
ab.fit(train_features, train_target)

knn = KNeighborsRegressor()#create a dictionary of all values we want to test for n_neighbors
params_knn = {'n_neighbors': np.arange(1, 25)}#use gridsearch to test all values for n_neighbors
knn_gs = GridSearchCV(knn, params_knn, cv=55)#fit model to training data
knn_gs.fit(train_features, train_target)

estimators=[('knn', knn_gs), ('rf', rf), ('lreg', lreg),('br',br),('ab',ab)]
voter = VotingRegressor(estimators)
voter.fit(train_features, train_target)



/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


VotingRegressor(estimators=[('knn',
                             GridSearchCV(cv=55,
                                          error_score='raise-deprecating',
                                          estimator=KNeighborsRegressor(algorithm='auto',
                                                                        leaf_size=30,
                                                                        metric='minkowski',
                                                                        metric_params=None,
                                                                        n_jobs=None,
                                                                        n_neighbors=5,
                                                                        p=2,
                                                                        weights='uniform'),
                                          iid='warn', n_jobs=None,
                                          param_grid={'n_neighbors': arr

# Tuning Hyperparameters for RandomForestRegressor

In [366]:
#pprint(rf.get_params()) #Checking hyper parameters of rf

#Creating our random grid to sample during fitting
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

pprint(random_grid)


{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [367]:
rrf = RandomForestRegressor() #Base RFR for use in our RandomizedSCV
rf_random = RandomizedSearchCV(estimator = rrf, 
                               param_distributions = random_grid, 
                               n_iter = 10, cv = 3, verbose = 2,
                               random_state = 42, n_jobs = -1)

rf_random.fit(train_features, train_target)



Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  5.6min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators='warn',
                                                   n_jobs=None, oob_score=False,
                                                   random_sta...


In [368]:
rf_random.best_params_   #Looking at best params

{'bootstrap': True,
 'max_depth': 50,
 'max_features': 'auto',
 'min_samples_leaf': 2,
 'min_samples_split': 5,
 'n_estimators': 2000}

In [369]:
best_random = rf_random.best_estimator_   #Setting new RSCV using best parameters
best_random.fit(train_features, train_target)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=50,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=2, min_samples_split=5,
                      min_weight_fraction_leaf=0.0, n_estimators=2000,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

# Predict and Measure Using Test 1

In [370]:
test_features = test_df[feature_cols] 

# impute missing values with medians
imp = SimpleImputer(missing_values=np.nan, strategy='median')
imp.fit(test_features)
imp.transform(test_features)


array([[1.0000e+00, 1.0000e+00, 7.0000e+00, ..., 3.1800e+01, 7.9781e+04,
        2.2830e+01],
       [2.0000e+00, 2.0000e+00, 6.0000e+00, ..., 3.1800e+01, 7.9781e+04,
        2.2830e+01],
       [2.0000e+00, 1.0000e+00, 6.0000e+00, ..., 3.1800e+01, 7.9781e+04,
        2.2830e+01],
       ...,
       [2.0000e+00, 2.0000e+00, 2.0000e+01, ..., 4.4200e+01, 3.6593e+04,
        4.2100e+01],
       [3.0000e+00, 2.0000e+00, 1.0000e+00, ..., 4.4000e+01, 7.6244e+04,
        5.8370e+01],
       [2.0000e+00, 1.0000e+00, 2.0000e+00, ..., 4.4000e+01, 7.6244e+04,
        5.8370e+01]])

In [371]:
# construct predictions 
test_df['predicted_rf'] = rf.predict(test_features)
print("MSE_rf", mean_squared_error(test_df['rent'], test_df['predicted_rf']))

test_df['predicted_gb'] = gb.predict(test_features)
print("MSE_gb", mean_squared_error(test_df['rent'], test_df['predicted_gb']))

test_df['predicted_ab'] = ab.predict(test_features)
print("MSE_ab", mean_squared_error(test_df['rent'], test_df['predicted_ab']))

test_df['predicted_br'] = br.predict(test_features)
print("MSE_br", mean_squared_error(test_df['rent'], test_df['predicted_br']))

test_df['predicted_lr'] = lreg.predict(test_features)
print("MSE_lreg", mean_squared_error(test_df['rent'], test_df['predicted_lr']))

test_df['predicted_knn'] = knn_gs.predict(test_features)
print("MSE_knn", mean_squared_error(test_df['rent'], test_df['predicted_knn']))

test_df['predicted_v'] = voter.predict(test_features)
print("MSE_voter", mean_squared_error(test_df['rent'], test_df['predicted_v']))

test_df['predicted_rscv_best'] = best_random.predict(test_features)   #Testing randomized search CV
print("MSE_Best_RSCV", mean_squared_error(test_df['rent'], test_df['predicted_rscv_best']))



MSE_rf 2369543.8237394583
MSE_gb 2281905.3433675137
MSE_ab 11656765.734535042
MSE_br 2339523.21823827
MSE_lreg 3318977.951345667
MSE_knn 2829891.9005977777
MSE_voter 2696418.8346465067
MSE_RSCV 2135752.289256745
MSE_Best_RSCV 2135752.289256745


##Display random sample for testing purposes

In [375]:
display(test_df[['rent','predicted_rf','predicted_gb','predicted_ab',
                 'predicted_br','predicted_lr','predicted_knn','predicted_v',
                 'predicted_rscv_best']].sample(5).transpose())

rental_id,7432930,7500238,7481374,7462291,7516789
rent,2649.000000,6000.000000,2550.000000,3000.000000,3995.000000
predicted_rf,2731.520000,5026.600000,2991.174242,3058.000000,4794.100000
predicted_gb,3360.668938,5934.583032,2681.004477,2561.386409,3810.472398
predicted_ab,5770.871489,7525.776929,4396.158076,8170.547341,5996.582886
predicted_br,2780.850000,4870.900000,2929.138462,3129.650000,3935.166667
predicted_lr,1918.878973,6796.012383,2870.848083,3119.768489,4054.982343
predicted_knn,2881.133333,5797.466667,3067.000000,2667.866667,3971.800000
predicted_v,3247.205585,5764.798995,3259.388291,4011.733714,4545.432520
predicted_rscv_best,2773.187420,5336.944449,2769.542952,2858.640844,4132.024563


#### Predict and Measure Using Test 1 (Hyper Tuned RF Regressor)

# Combine Data, Predict Values for Test 3

In [377]:
master_df = train_df.append(test_df, sort=False)

#impute missing values with imputer
imp = SimpleImputer(missing_values=np.nan, strategy='median')
imp.fit(master_features)
imp.transform(master_features)

master_features = master_df[feature_cols].fillna(master_df[feature_cols].median(), axis=0)

master_target = master_df['rent']

rf.fit(master_features, master_target)
gb.fit(master_features, master_target)
ab.fit(master_features, master_target)
br.fit(master_features, master_target)
lreg.fit(master_features, master_target)

params_knn = {'n_neighbors': np.arange(1, 30)}#use gridsearch to test all values for n_neighbors
knn_gs = GridSearchCV(knn, params_knn, cv=20)#fit model to training data
knn_gs.fit(master_features, master_target)

estimators=[('knn', knn_gs), ('rf', rf), ('lreg', lreg),('br',br),('ab',ab),('best_random',best_random)]
voter = VotingRegressor(estimators)
voter.fit(master_features, master_target)

best_random.fit(master_features, master_target)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=50,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=2, min_samples_split=5,
                      min_weight_fraction_leaf=0.0, n_estimators=2000,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

#### Best Randomized Search CV (test2)

In [378]:
submit2_features = submit2_df[feature_cols].fillna(master_df[feature_cols].median(), axis=0)
submit2_df['predictions'] = best_random.predict(submit2_features)
submit2_df['predictions'].to_csv('submission_rf.csv', header=True)

submit2_df['fake_rent'] = np.ones(submit2_df['predictions'].shape) * master_target.median()
print("MVE_best_rf: ", mean_squared_error(submit2_df['predictions'], submit2_df['fake_rent']))

MVE_best_rf:  5430249.638488843


#### Random Forest Regressor (test2)

In [379]:
submit2_features = submit2_df[feature_cols].fillna(master_df[feature_cols].median(), axis=0)
submit2_df['predictions'] = rf.predict(submit2_features)
submit2_df['predictions'].to_csv('submission_rf.csv', header=True)

submit2_df['fake_rent'] = np.ones(submit2_df['predictions'].shape) * master_target.median()
print("MVE_rf: ", mean_squared_error(submit2_df['predictions'], submit2_df['fake_rent']))

MVE_rf:  5943211.154572495


#### GradientBoosting Regressor (test2)

In [380]:
submit2_features = submit2_df[feature_cols].fillna(master_df[feature_cols].median(), axis=0)
submit2_df['predictions'] = rf.predict(submit2_features)
submit2_df['predictions'].to_csv('sample_submission2.csv', header=True)

submit2_df['fake_rent'] = np.ones(submit2_df['predictions'].shape) * master_target.median()
print("MVE_gb: ", mean_squared_error(submit2_df['predictions'], submit2_df['fake_rent']))

MVE_gb:  5943211.154572495


#### ABoost Regressor (test2)

In [381]:
submit2_features = submit2_df[feature_cols].fillna(master_df[feature_cols].median(), axis=0)
submit2_df['predictions'] = ab.predict(submit2_features)
submit2_df['predictions'].to_csv('submission_ab.csv', header=True)

submit2_df['fake_rent'] = np.ones(submit2_df['predictions'].shape) * master_target.median()
print("MVE_ab: ", mean_squared_error(submit2_df['predictions'], submit2_df['fake_rent']))

MVE_ab:  6240317.591662033


#### Bagged Regressor (test2)

In [382]:
submit2_features = submit2_df[feature_cols].fillna(master_df[feature_cols].median(), axis=0)
submit2_df['predictions'] = br.predict(submit2_features)
submit2_df['predictions'].to_csv('submission_br.csv', header=True)

submit2_df['fake_rent'] = np.ones(submit2_df['predictions'].shape) * master_target.median()
print("MVE_br: ", mean_squared_error(submit2_df['predictions'], submit2_df['fake_rent']))

MVE_br:  5579690.712832238


#### Linear Regression (test2)

In [383]:
submit2_features = submit2_df[feature_cols].fillna(master_df[feature_cols].median(), axis=0)
submit2_df['predictions'] = lreg.predict(submit2_features)
submit2_df['predictions'].to_csv('sample_submission_lreg.csv', header=True)

submit2_df['fake_rent'] = np.ones(submit2_df['predictions'].shape) * master_target.median()
print("MVE_lreg: ", mean_squared_error(submit2_df['predictions'], submit2_df['fake_rent']))

MVE_lreg:  4482816.462852407


#### KNN Regressor (test2)

In [385]:
submit2_features = submit2_df[feature_cols].fillna(master_df[feature_cols].median(), axis=0)
submit2_df['predictions'] = knn_gs.predict(submit2_features)
submit2_df['predictions'].to_csv('sample_submission_knn.csv', header=True)

submit2_df['fake_rent'] = np.ones(submit2_df['predictions'].shape) * master_target.median()
print("MVE_knn: ", mean_squared_error(submit2_df['predictions'], submit2_df['fake_rent']))


MVE_knn:  5440391.373351563


### Voting Regressor (test2)

In [386]:
submit2_features = submit2_df[feature_cols].fillna(master_df[feature_cols].median(), axis=0)
submit2_df['predictions'] = voter.predict(submit2_features)
submit2_df['predictions'].to_csv('submission_vote.csv', header=True)

submit2_df['fake_rent'] = np.ones(submit2_df['predictions'].shape) * master_target.median()
print("MVE_vote: ", mean_squared_error(submit2_df['predictions'], submit2_df['fake_rent']))

MVE_vote:  5153130.392821633
